In [82]:
# --- IMPORTS ---
import cv2
from ultralytics import YOLO
import easyocr
import numpy as np
import re

In [83]:
# --- INITIALIZATION ---

# Load the YOLO model
# Replace 'path/best.pt' with the actual path to the trained model file
model = YOLO('35epoch.pt')

# Initialize EasyOCR Reader
reader = easyocr.Reader(['en'], gpu=True) # Use gpu=False if there is no CUDA-enabled GPU

# Open the video file
video_path = 'test2.MOV'
cap = cv2.VideoCapture(video_path)

In [84]:
# --- VIDEO WRITER SETUP ---
# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create VideoWriter object
output_path = 'output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Codec for .mp4 file
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

**This code part detect 100% plate**

In [85]:
# # --- VIDEO PROCESSING LOOP ---

# while cap.isOpened():
#     ret, frame = cap.read()
#     if not ret:
#         break

#     # Perform object detection
#     results = model(frame, stream=True)

#     # Process results
#     for result in results:
#         boxes = result.boxes
#         for box in boxes:
#             # Check if the detected object is a plate number (class index 1)
#             if int(box.cls[0]) == 1:
#                 x1, y1, x2, y2 = map(int, box.xyxy[0])
#                 plate_crop = frame[y1:y2, x1:x2]

#                 try:
#                     # Use EasyOCR to read text
#                     plate_text_results = reader.readtext(plate_crop, detail=0, paragraph=False)
#                     plate_text = " ".join(plate_text_results).upper().replace(" ", "") if plate_text_results else "READING..."
#                 except Exception as e:
#                     plate_text = "ERROR"
                
#                 # Draw results on the frame
#                 cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
#                 (text_width, text_height), baseline = cv2.getTextSize(plate_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
#                 cv2.rectangle(frame, (x1, y1 - text_height - 10), (x1 + text_width, y1 - 5), (0, 255, 0), -1)
#                 cv2.putText(frame, plate_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

#     # --- WRITE FRAME TO VIDEO FILE ---
#     out.write(frame)

#     # Display the processed frame (optional, can be commented out for faster processing)
#     # cv2.imshow('YOLO + OCR Vehicle Recognition', frame)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Clean up
# cap.release()
# out.release()
# cv2.destroyAllWindows()

**This code part detect 70% plate**

In [86]:
# # --- VIDEO PROCESSING LOOP ---

# while cap.isOpened():
#     ret, frame = cap.read()
#     if not ret:
#         break

#     # Perform object detection
#     results = model(frame, stream=True)

#     # Process results
#     for result in results:
#         boxes = result.boxes
#         for box in boxes:
#             if int(box.cls[0]) == 2:
#                 x1, y1, x2, y2 = map(int, box.xyxy[0])

#                 # Crop the top 70% of the plate's bounding box
#                 box_height = y2 - y1
#                 y2_top_part = y1 + int(box_height * 0.70)
#                 plate_crop = frame[y1:y2_top_part, x1:x2]

#                 try:
#                     # Use EasyOCR to read text from the cropped top part
#                     plate_text_results = reader.readtext(plate_crop, detail=0, paragraph=False)
#                     plate_text = " ".join(plate_text_results).upper().replace(" ", "") if plate_text_results else "READING..."
#                 except Exception as e:
#                     plate_text = "ERROR"

#                 # Draw the original, full bounding box on the frame
#                 cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                
#                 # Draw the text background and the recognized text
#                 (text_width, text_height), baseline = cv2.getTextSize(plate_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
#                 cv2.rectangle(frame, (x1, y1 - text_height - 10), (x1 + text_width, y1 - 5), (0, 255, 0), -1)
#                 cv2.putText(frame, plate_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

#     # --- WRITE FRAME TO VIDEO FILE ---
#     out.write(frame)

#     # Display the processed frame (optional)
#     # cv2.imshow('YOLO + OCR Vehicle Recognition', frame)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Clean up
# cap.release()
# out.release()
# cv2.destroyAllWindows()

**This code part detect 70% plate as license number of plate and the 30% as exp date**

In [87]:
# # --- VIDEO PROCESSING LOOP ---

# while cap.isOpened():
#     ret, frame = cap.read()
#     if not ret:
#         break

#     results = model(frame, stream=True)

#     for result in results:
#         boxes = result.boxes
#         for box in boxes:
#             # Check for 'LicensePlate' which has class index 2
#             if int(box.cls[0]) == 2:
#                 x1, y1, x2, y2 = map(int, box.xyxy[0])
                
#                 try:
#                     # 1. Split the bounding box
#                     box_height = y2 - y1
#                     split_point = y1 + int(box_height * 0.70)
                    
#                     plate_crop = frame[y1:split_point, x1:x2]
#                     expiry_crop = frame[split_point:y2, x1:x2]

#                     # 2. OCR on the plate number
#                     plate_text_results = reader.readtext(plate_crop, detail=0, paragraph=False)
#                     plate_text = "".join(plate_text_results).upper().replace(" ", "") if plate_text_results else "..."

#                     # 3. Preprocess the expiry date crop to improve clarity
#                     if expiry_crop.shape[0] > 0 and expiry_crop.shape[1] > 0:
#                         expiry_gray = cv2.cvtColor(expiry_crop, cv2.COLOR_BGR2GRAY)
#                         expiry_resized = cv2.resize(expiry_gray, None, fx=2.5, fy=2.5, interpolation=cv2.INTER_CUBIC)
#                         _, expiry_processed = cv2.threshold(expiry_resized, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
#                     else:
#                         expiry_processed = None

#                     # 4. OCR using an 'allowlist' to force number recognition
#                     raw_expiry_text = ""
#                     if expiry_processed is not None:
#                         expiry_text_results = reader.readtext(
#                             expiry_processed,
#                             detail=0,
#                             paragraph=False,
#                             allowlist='0123456789.'
#                         )
#                         raw_expiry_text = "".join(expiry_text_results)

#                     # 5. Keep only the digits from the now-cleaner text
#                     digits_only = re.sub(r'\D', '', raw_expiry_text)
                    
#                     # 6. Final validation
#                     if len(digits_only) == 4:
#                         expiry_text = f"{digits_only[:2]}.{digits_only[2:]}"
#                     else:
#                         expiry_text = "INVALID"
                
#                 except Exception as e:
#                     plate_text = "ERROR"
#                     expiry_text = "ERROR"

#                 # 7. Draw results
#                 cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
#                 display_text_plate = f"Plate: {plate_text}"
#                 display_text_expiry = f"Expires: {expiry_text}"
                
#                 (w_plate, h_plate), _ = cv2.getTextSize(display_text_plate, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
#                 (w_expiry, h_expiry), _ = cv2.getTextSize(display_text_expiry, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
#                 max_w = max(w_plate, w_expiry)
#                 cv2.rectangle(frame, (x1, y1 - (h_plate + h_expiry) - 20), (x1 + max_w + 10, y1), (0, 255, 0), -1)

#                 cv2.putText(frame, display_text_plate, (x1 + 5, y1 - h_expiry - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
#                 cv2.putText(frame, display_text_expiry, (x1 + 5, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)

#     out.write(frame)
#     # cv2.imshow('License Plate Recognition', frame)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # --- CLEAN UP ---
# cap.release()
# out.release()
# cv2.destroyAllWindows()

In [88]:
# --- HELPER FUNCTIONS ---

def preprocess_expiry_crop(crop):
    """The most effective preprocessing function for the expiry date."""
    if crop is None or crop.shape[0] < 5 or crop.shape[1] < 10:
        return None
    
    gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
    
    # Slightly enlarge the size for clarity
    scale = max(2.0, 50.0 / gray.shape[0])
    resized = cv2.resize(gray, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
    
    # Apply threshold to make the text black and white (binarize)
    _, thresh = cv2.threshold(resized, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    return thresh

def extract_and_validate_date(text):
    """A simpler and faster function to validate MM.YY date format."""
    if not text:
        return "INVALID"
    
    # Get all digits from the text
    digits = re.sub(r'\D', '', text)
    
    # If there are 4 digits (for MMYY)
    if len(digits) == 4:
        month_str, year_str = digits[:2], digits[2:]
        # Validate that the month is between 01 and 12
        try:
            if 1 <= int(month_str) <= 12:
                return f"{month_str}.{year_str}"
        except ValueError:
            pass # Ignore if conversion fails
            
    return "INVALID"

In [89]:
# --- VIDEO PROCESSING LOOP ---

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, stream=True)

    for result in results:
        boxes = result.boxes
        for box in boxes:
            # Ensure class_id 2 is for 'LicensePlate'
            if int(box.cls[0]) == 2:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                
                plate_text = "..."
                expiry_text = "INVALID"
                
                try:
                    # 1. Split the license plate and expiry date areas
                    box_height = y2 - y1
                    split_point = y1 + int(box_height * 0.70)
                    
                    plate_crop = frame[y1:split_point, x1:x2]
                    expiry_crop = frame[split_point:y2, x1:x2]

                    # 2. OCR on the plate number (remains the same)
                    plate_text_results = reader.readtext(plate_crop, detail=0, paragraph=False)
                    plate_text = "".join(plate_text_results).upper().replace(" ", "") if plate_text_results else "..."

                    # 3. Process the expiry date (ONLY ONCE)
                    processed_expiry = preprocess_expiry_crop(expiry_crop)
                    
                    if processed_expiry is not None:
                        # 4. Run OCR ONLY ONCE with the best configuration
                        ocr_results = reader.readtext(
                            processed_expiry,
                            detail=0,
                            paragraph=False,
                            allowlist='0123456789.' # Force OCR to only read digits and dots
                        )
                        raw_text = "".join(ocr_results) if ocr_results else ""
                        
                        # 5. Validate the date with the optimized function
                        expiry_text = extract_and_validate_date(raw_text)
                
                except Exception as e:
                    plate_text = "ERROR"
                    expiry_text = "ERROR"
                    # print(f"Error: {e}") # Uncomment for debugging

                # 6. Draw the results on the frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                display_text_plate = f"Plate: {plate_text}"
                display_text_expiry = f"Expires: {expiry_text}"
                
                (w_plate, h_plate), _ = cv2.getTextSize(display_text_plate, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
                (w_expiry, h_expiry), _ = cv2.getTextSize(display_text_expiry, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
                max_w = max(w_plate, w_expiry)
                cv2.rectangle(frame, (x1, y1 - (h_plate + h_expiry) - 20), (x1 + max_w + 10, y1), (0, 255, 0), -1)

                cv2.putText(frame, display_text_plate, (x1 + 5, y1 - h_expiry - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
                cv2.putText(frame, display_text_expiry, (x1 + 5, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)

    out.write(frame)
    # cv2.imshow('License Plate Recognition', frame) # Uncomment to see a preview during processing

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


0: 384x640 2 LicensePlates, 3 Motorcycles, 60.3ms
Speed: 10.3ms preprocess, 60.3ms inference, 10.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 LicensePlates, 3 Motorcycles, 63.7ms
Speed: 4.4ms preprocess, 63.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 LicensePlates, 3 Motorcycles, 43.7ms
Speed: 2.3ms preprocess, 43.7ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 LicensePlates, 3 Motorcycles, 41.9ms
Speed: 2.7ms preprocess, 41.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 LicensePlates, 3 Motorcycles, 34.2ms
Speed: 1.9ms preprocess, 34.2ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 LicensePlates, 3 Motorcycles, 34.5ms
Speed: 1.8ms preprocess, 34.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 LicensePlates, 3 Motorcycles, 36.1ms
Speed: 1.9ms preprocess, 36.1ms inference, 0.5ms postproces

In [90]:
print(f"Processed video saved to: {output_path}")

Processed video saved to: output_video.mp4
